In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import prettytable as pt

np.random.seed(100) # Setting a random seed for reproducibility
np.set_printoptions(precision=3, suppress=True) # Setting print options for NumPy arrays

In [2]:
class ascending_bid_auction:
    def __init__(self, v, r, ϵ):
        """
        A class that simulates an ascending bid auction for houses.
        Given buyers' value matrix, sellers' reservation prices and minimum increment of bid prices,
        this class can execute an ascending bid auction and present information round by round until the end.
        Parameters:
        ----------
        v: 2 dimensional value matrix
        r: np.array of reservation prices
        ϵ: minimum increment of bid price
        """
        self.v = v.copy()
        self.n,self.m = self.v.shape
        self.r = r
        self.ϵ = ϵ
        self.p = r.copy()
        self.buyer_list = np.arange(self.m)
        self.house_list = np.arange(self.n)
        self.bid_dict_history = []
        self.allocation_history = []
        self.winner_history = []
        self.loser_history = []


    def find_argmax_with_randomness(self, v):
        n,m = v.shape
        index_array = np.arange(n)
        result=[]

        #complete this section
        # Randomly select one of the maximum values
        result = [np.random.choice(index_array[v[:, i] == v[:, i].max()]) for i in range(m)]
        
        return np.array(result)  # Return the array of randomly selected indices


    def check_kick_off_condition(self):
        # we convert the price vector to a matrix in the same shape as value matrix to facilitate subtraction
        #complete this section
        er = self.ϵ + self.r
        p_start = er[:, None] @ np.ones(self.m)[None, :] # Convert the price vector to a matrix for subtraction
        self.surplus_value = self.v - p_start # Calculate the surplus value for each buyer and house
        buyer_decision = (self.surplus_value > 0).any(axis=0) # Check if there is any buyer with positive surplus value
        
        return buyer_decision.any()  # Return True if there is at least one buyer with positive surplus value, otherwise False


    def submit_initial_bid(self):
        #complete this section
        #  intend to find the optimal choice of each buyer 
        ep = self.ϵ + self.p
        p_start_mat = ep[:, None] @ np.ones(self.m)[None, :] # Convert the price vector to a matrix for subtraction
        self.surplus_value = self.v - p_start_mat # Calculate the surplus value for each buyer and house

        #  only care about active buyers who have positve surplus values
        pos_svals = self.surplus_value > 0
        pos_buyers = pos_svals.any(axis=0)  # Check which buyers have positive surplus values
        ls_pos_buyers = self.buyer_list[pos_buyers]  # Get the list of active buyers
        pos_buyers_sval = self.surplus_value[:, pos_buyers]  # Get the surplus values of active buyers
        pos_buyers_choices = self.find_argmax_with_randomness(pos_buyers_sval)  # Find the optimal choice of each active buyer

        #  only retain the unique house index because prices increase once at one round
        bid = list(set(pos_buyers_choices))  # Retain only the unique house indices because prices increase once at each round
        self.p[bid] += self.ϵ  # Increase the prices of the houses being bid on

        bid_dict = {}
        for number in bid:
            bid_dict[number] = ls_pos_buyers[pos_buyers_choices == number]  # Store the buyer indices bidding on each house
        self.bid_dict_history.append(bid_dict)  # Store the bid information in the history
        
        
        print('The bid information is')
        ymtb = pt.PrettyTable()
        ymtb.field_names = ['House Number', *bid_dict.keys()]
        ymtb.add_row(['Buyer', *bid_dict.values()])
        print(ymtb)
        
        print('The bid prices for houses are')
        ymtb = pt.PrettyTable()
        ymtb.field_names = ['House Number', *self.house_list]
        ymtb.add_row(['Price', *self.p])
        print(ymtb)
        
        self.winner_list=[np.random.choice(bid_dict[i]) for i in bid_dict.keys()]
        self.winner_history.append(self.winner_list)
        
        self.allocation = {number:[winner] for number,winner in zip(bid_dict.keys(),self.winner_list)}
        self.allocation_history.append(self.allocation)
        
        loser_set = set(self.buyer_list).difference(set(self.winner_list))
        self.loser_list = list(loser_set)
        self.loser_history.append(self.loser_list)
        
        print('The winners are')
        print(self.winner_list)

        print('The losers are')
        print(self.loser_list)
        print('\n')


    def check_terminal_condition(self):
        loser_num = len(self.loser_list)

        if loser_num == 0:
            print('The auction ends because every buyer gets one house.')
            print('\n')
            return True

        p_mat = (self.ϵ + self.p)[:,None] @ np.ones(loser_num)[None,:]
        self.loser_surplus_value = self.v[:,self.loser_list] - p_mat
        self.loser_decision = (self.loser_surplus_value > 0).any(axis = 0)

        return ~(self.loser_decision.any())


    def submit_bid(self):
        bid_dict = self.allocation_history[-1].copy()  #  only record the bid info of winner
        # retain the unique house index and increasing the corresponding bid price
        # we record the bidding information from active losers
        # we update the bidding information according to the bidding from actice losers

        #complete this section

        bid_dict = self.allocation_history[-1].copy()  # we only record the bid info of winner

        losers_count = len(self.loser_list)  # Number of losers
        price_matrix = (self.ϵ + self.p)[:, None] @ np.ones(losers_count)[None, :]  # Bid price matrix for losers
        self.loser_surplus_value = self.v[:, self.loser_list] - price_matrix  # Surplus value matrix for losers
        self.loser_decision = (self.loser_surplus_value > 0).any(axis=0)  # Bidding decisions of losers

        ls_losers = np.array(self.loser_list)[self.loser_decision]  # List of active losers
        losers_svals = self.loser_surplus_value[:, self.loser_decision]  # Surplus value of active losers
        losers_choices = self.find_argmax_with_randomness(losers_svals)  # Optimal choice of active losers


        # we retain the unique house index and increasing the corresponding bid price
        bid = list(set(losers_choices))  
        self.p[bid] += self.ϵ

        # we record the bidding information from active losers
        bid_dict_active_loser = {}
        for number in bid:
            bid_dict_active_loser[number] = ls_losers[losers_choices == number]

        # we update the bidding information according to the bidding from actice losers
        for number in bid_dict_active_loser.keys():
            bid_dict[number] = bid_dict_active_loser[number]
        self.bid_dict_history.append(bid_dict)

        #do not change below
        print('The bid information is')
        ymtb = pt.PrettyTable()
        ymtb.field_names = ['House Number', *bid_dict.keys()]
        ymtb.add_row(['Buyer', *bid_dict.values()])
        print(ymtb)

        print('The bid prices for houses are')
        ymtb = pt.PrettyTable()
        ymtb.field_names = ['House Number', *self.house_list]
        ymtb.add_row(['Price', *self.p])
        print(ymtb)

        self.winner_list=[np.random.choice(bid_dict[i]) for i in bid_dict.keys()]
        self.winner_history.append(self.winner_list)

        self.allocation = {number:[winner] for number,winner in zip(bid_dict.keys(),self.winner_list)}
        self.allocation_history.append(self.allocation)

        loser_set = set(self.buyer_list).difference(set(self.winner_list))
        self.loser_list = list(loser_set)
        self.loser_history.append(self.loser_list)

        print('The winners are')
        print(self.winner_list)

        print('The losers are')
        print(self.loser_list)
        print('\n')


    def start_auction(self):
        print('The Ascending Bid Auction for Houses')
        print('\n')

        print('Basic Information: %d houses, %d buyers'%(self.n, self.m))

        print('The valuation matrix is as follows')
        ymtb = pt.PrettyTable()
        ymtb.field_names = ['Buyer Number', *(np.arange(self.m))]
        for ii in range(self.n):
            ymtb.add_row(['House %d'%(ii), *self.v[ii,:]])
        print(ymtb)

        print('The reservation prices for houses are')
        ymtb = pt.PrettyTable()
        ymtb.field_names = ['House Number', *self.house_list]
        ymtb.add_row(['Price', *self.r])
        print(ymtb)
        print('The minimum increment of bid price is %.2f' % self.ϵ)
        print('\n')

        ctr = 1
        if self.check_kick_off_condition():
            print('Auction starts successfully')
            print('\n')
            print('Round %d'% ctr)

            self.submit_initial_bid()

            while True:
                if self.check_terminal_condition():
                    print('Auction ends')
                    print('\n')

                    print('The final result is as follows')
                    print('\n')
                    print('The allocation plan is')
                    ymtb = pt.PrettyTable()
                    ymtb.field_names = ['House Number', *self.allocation.keys()]
                    ymtb.add_row(['Buyer', *self.allocation.values()])
                    print(ymtb)

                    print('The bid prices for houses are')
                    ymtb = pt.PrettyTable()
                    ymtb.field_names = ['House Number', *self.house_list]
                    ymtb.add_row(['Price', *self.p])
                    print(ymtb)

                    print('The winners are')
                    print(self.winner_list)

                    print('The losers are')
                    print(self.loser_list)

                    self.house_unsold_list = list(set(self.house_list).difference(set(self.allocation.keys())))
                    print('The houses unsold are')
                    print(self.house_unsold_list)

                    self.total_revenue = self.p[list(self.allocation.keys())].sum()
                    print('The total revenue is %.2f' % self.total_revenue)

                    break

                ctr += 1
                print('Round %d'% ctr)
                self.submit_bid()

            #  compute the surplus matrix S and the quantity matrix X as required in 1.1
            # we compute the surplus matrix S and the quantity matrix X as required in 1.1
            self.S = np.zeros((self.n, self.m)) # Initializing the surplus matrix (S) with zeros
            # Computing the surplus matrix (S) by subtracting the bid prices from the corresponding buyer values
            for i,j in self.allocation.items():
                self.S[i,j] = self.v[i,j] - self.p[i]
                
            # Initializing the quantity matrix (Q) with zeros, with an additional column to record unsold houses
            self.Q = np.zeros((self.n, self.m + 1))
            # Filling the quantity matrix (Q) with ones to indicate sold houses
            for i,j in self.allocation.items(): 
                self.Q[i,j] = 1
            # Marking the unsold houses by setting the last column of the quantity matrix (Q) to ones
            for i in self.house_unsold_list:
                self.Q[i,-1] = 1

            #  sort the allocation result by the house number
            # we sort the allocation result by the house number
            house_sold_list = list(self.allocation.keys())
            house_sold_list.sort()
            
            dict_temp = {}
            for i in house_sold_list:
                dict_temp[i] = self.allocation[i]
            self.allocation = dict_temp

        else:
            print('The auction can not start because of high reservation prices')

In [3]:
v = np.array([[8,5,9,4],[4,11,7,4],[9,7,6,4]])
r = np.array([2,1,0])
ϵ = 1

auction_1 = ascending_bid_auction(v, r, ϵ)

auction_1.start_auction()

The Ascending Bid Auction for Houses


Basic Information: 3 houses, 4 buyers
The valuation matrix is as follows
+--------------+---+----+---+---+
| Buyer Number | 0 | 1  | 2 | 3 |
+--------------+---+----+---+---+
|   House 0    | 8 | 5  | 9 | 4 |
|   House 1    | 4 | 11 | 7 | 4 |
|   House 2    | 9 | 7  | 6 | 4 |
+--------------+---+----+---+---+
The reservation prices for houses are
+--------------+---+---+---+
| House Number | 0 | 1 | 2 |
+--------------+---+---+---+
|    Price     | 2 | 1 | 0 |
+--------------+---+---+---+
The minimum increment of bid price is 1.00


Auction starts successfully


Round 1
The bid information is
+--------------+-----+-----+-------+
| House Number |  0  |  1  |   2   |
+--------------+-----+-----+-------+
|    Buyer     | [2] | [1] | [0 3] |
+--------------+-----+-----+-------+
The bid prices for houses are
+--------------+---+---+---+
| House Number | 0 | 1 | 2 |
+--------------+---+---+---+
|    Price     | 3 | 2 | 1 |
+--------------+---+---+---+
T